<a href="https://colab.research.google.com/github/freealise/freealise/blob/main/opencv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ffmpy
import IPython.display as display
from ffmpy import FFmpeg
import numpy as np
import cv2 as cv

img1 = cv.imread("cube_origs.jpg", cv.IMREAD_UNCHANGED)
img2 = cv.imread("cube_origs.jpg", cv.IMREAD_UNCHANGED)
img = np.concatenate((img1, img2), axis=1)
cv.imwrite("cube.jpg", img)

ff = FFmpeg(
    inputs={'cube.jpg': None },
    outputs={'output.jpg': '-vf v360=dfisheye:equirect:ih_fov=180:iv_fov=180:yaw=90' }
)
ff.run()

#video="output.jpg"
#display.display(display.Markdown(F"""![]({video})"""))

(None, None)

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
